In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import distributions as dist
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

In [ ]:
iris = datasets.load_iris()
X = iris['data']
y = iris['target']

J = np.random.uniform(size=X.shape)
thresh = 0.2
J[J < thresh] = 0
J[J >= thresh] = 1

X.shape, J.shape, y.shape, set(y)


In [ ]:
X_train, X_test, J_train, J_test, y_train, y_test = train_test_split(X, J, y, test_size=0.33, random_state=42)

In [ ]:
ds_train = TensorDataset(
    torch.Tensor(X_train), 
    torch.Tensor(J_train),
    torch.Tensor(y_train).long()
)

ds_test = TensorDataset(
    torch.Tensor(X_test), 
    torch.Tensor(J_test),
    torch.Tensor(y_test).long()
)

batch_size=16
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_test = DataLoader(ds_test, batch_size, shuffle=True)

In [ ]:
class Reshaper(nn.Module):
    def __init__(self, out_size):
        super().__init__()

        self.out_size = out_size
        
    def forward(self, x):
        return x.view(*self.out_size)

In [ ]:
class IrisInpainter(
    nn.Module
):
    def __init__(self, n_mixes: int = 1, in_size: int = 4, a_width = 3):
        super().__init__()

        
        self.extractor = nn.Sequential(
            nn.Linear(in_size * 2, 10),
            nn.ReLU(),
            nn.Linear(10, 20),
            nn.ReLU(),
        )
        
        self.a_extractor = nn.Sequential(
            nn.Linear(20, in_size * n_mixes * a_width),
            Reshaper((-1, n_mixes, a_width, in_size,)) # * L, we don't want 1x4 vector but L x4 matrix))
        )
        self.m_extractor = nn.Sequential(
            nn.Linear(20, n_mixes * in_size),
            Reshaper((-1, n_mixes, in_size))

        )
        
        self.d_extractor = nn.Sequential(
            nn.Linear(20, n_mixes * in_size),
            Reshaper((-1, n_mixes, in_size))

        )
        
        self.p_extractor = nn.Sequential(
            nn.Linear(20, n_mixes),
            nn.Softmax()
        ) # omit this, let's say p = 1 for now

    def forward(self, X, J):
        X_masked = X * J
        X_J = torch.cat([X_masked, J], dim=1)
        features = self.extractor(X_J)
        m = self.m_extractor(features)
        d = self.d_extractor(features)
        p = self.p_extractor(features)
        a = self.a_extractor(features)
        
        return  p, m, a, d

In [ ]:

def nll_loss(X, J, P, M, A, D) -> torch.autograd.Variable:
    zipped = zip(X, J, P, M, A, D)
    losses = []
    
    
    for i, (x, j, p, m, a, d) in enumerate(zipped):
        mask_inds = (j==0).nonzero().squeeze()
        x_masked = torch.index_select(x, 0, mask_inds)
        a_masked = torch.index_select(a, 2, mask_inds)
        m_masked, d_masked = [
            torch.index_select(t, 1, mask_inds)
            for t in [m, d]
        ]
        
        for (p_i, m_i, d_i, a_i) in zip(p, m_masked, d_masked, a_masked):
            if a_i.shape[1] > 0:
                cov = (a_i.transpose(0,1) @ a_i) + torch.diag(d_i ** 2)
                mvn_d = dist.MultivariateNormal(m_i, cov) # calculate this manually
                l = - mvn_d.log_prob(x_masked) 
                losses.append(l)
            else:
                losses.append(torch.tensor(0.0, requires_grad=True)) #.cuda())
    return torch.stack(losses).mean()

In [ ]:
inpainter = IrisInpainter()
# inpainter = inpainter.cuda()
opt = optim.Adam(inpainter.parameters(), lr=4e-3)
n_epochs = 100
train_hist = []
val_hist = []
for e in tqdm(range(n_epochs)):
    inpainter.train()
    train_losses = [] 
    for i, (x, j, y) in enumerate(dl_train):
        inpainter.zero_grad()
#         x, j = x.cuda(), j.cuda()
        p, m, a, d = inpainter(x, j)
        loss = nll_loss(x, j, p, m, a, d)
        loss.backward()
        opt.step()
        train_losses.append(loss)
    train_hist.append(torch.stack(train_losses).mean())
    
    inpainter.eval()
    val_losses = []
    for i, (x, j, y) in enumerate(dl_test):
        inpainter.zero_grad()
#         x, j = x.cuda(), j.cuda()

        p, m, a, d = inpainter(x, j)
        loss = nll_loss(x, j, p, m, a, d)
        val_losses.append(loss)
    val_hist.append(torch.stack(val_losses).mean())

#     print(train_hist[-1], val_losses[-1])

plt.plot(list(range(n_epochs)), train_hist, label="train")
plt.plot(list(range(n_epochs)), val_hist, label="val")
plt.legend()
print(val_losses[-1])


In [ ]:
classifier = MLPClassifier((10, 20, 10, 4), learning_rate_init=4e-3, max_iter=1000)
classifier.fit(X_train, y_train)
accuracy_score(classifier.predict(X_test), y_test)

In [ ]:
X_test_masked = X_test * J_test
# X_test_masked[:, 0][J_test[:,0] == 0] = X_train[:,0].mean()
# X_test_masked[:, 1][J_test[:,1] == 0] = X_train[:,1].mean()
# X_test_masked[:, 2][J_test[:,2] == 0] = X_train[:,2].mean()
# X_test_masked[:, 3][J_test[:,3] == 0] = X_train[:,3].mean()


accuracy_score(classifier.predict(X_test_masked), y_test)
# print(X_train.mean(axis=0))
# X_test_masked

In [ ]:
P_test, M_test, A_test, D_test = inpainter(torch.Tensor(X_test_masked), torch.Tensor(J_test))

X_test_inpainted = X_test_masked.copy()
X_test_inpainted[J_test == 0] = M_test.detach().cpu().numpy()[:, 0][J_test == 0]

# X_test_inpainted = M_test.detach().cpu().numpy()[:, 0]
print("X_test[0]", X_test[0]), 
print("X_test_masked[0]", X_test_masked[0]) 
print("M_test.detach().numpy()[0, 0]", M_test.detach().cpu().numpy()[0, 0]) 
print("X_test_inpainted[0]", X_test_inpainted[0])

accuracy_score(classifier.predict(X_test_inpainted), y_test)

In [ ]:
P_test, M_test, A_test, D_test = inpainter(torch.Tensor(X_test_masked), torch.Tensor(J_test))
#
nll_loss(torch.tensor(X_test).float(), torch.tensor(J_test).float(), P_test, M_test, A_test, D_test), 


In [ ]:
A_test

In [ ]:
nll_loss(torch.tensor(X_test).float(), torch.tensor(J_test).float(), P_test, torch.tensor(X_test).float().reshape(-1, 1, 4), A_test, D_test), 
